### 1. Kaggle Evaluation Dataset Preprocessing

#### 1.1 Unzip datasets in each folder
- We downloaded datasets using Kaggle's API, resulting in 5,221 folders. Each folder contains a `dataset-metadata.json` file with metadata in the following structure:
  ```
{
  "id":
  "id_no":
  "datasetSlugNullable":
  "ownerUserNullable":
  "usabilityRatingNullable":
  "titleNullable":
  "subtitleNullable":
  "descriptionNullable":
  "datasetId":
  "datasetSlug":
  "hasDatasetSlug":
  "ownerUser":
  "hasOwnerUser":
  "usabilityRating":
  "hasUsabilityRating":
  "totalViews":
  "totalVotes":
  "totalDownloads":
  "title":
  "hasTitle":
  "subtitle":
  "hasSubtitle":
  "description":
  "hasDescription":
  "isPrivate":
  "keywords":
  "licenses":
  "collaborators":
  "data":
}
```
- Each folder also contains a zip file with the corresponding datasets. Our first step is to iterate through all 5,221 folders and unzip all the datasets.

In [ ]:
import os
import zipfile

In [ ]:
# Directory containing the folders with zip files
base_directory = os.path.join(os.getcwd(), "kaggle")

In [ ]:
# # Iterate over all folders in the base directory
# for folder_name in os.listdir(base_directory):
#     folder_path = os.path.join(base_directory, folder_name)
    
#     # Check if it's a directory
#     if os.path.isdir(folder_path):
#         # Look for zip files in the directory
#         for file_name in os.listdir(folder_path):
#             if file_name.endswith(".zip"):
#                 zip_path = os.path.join(folder_path, file_name)
                
#                 # Try to unzip the file
#                 try:
#                     with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#                         zip_ref.extractall(folder_path)
#                     print(f"Unzipped: {zip_path}")
#                 except zipfile.BadZipFile as e:
#                     print(f"Failed to unzip {zip_path}: {e}")

# print("All zip files have been unzipped.")

#### 1.2 Check the files under each folder downloaded from Kaggle

- Original unfiltered # of files in total: 12,533

In [ ]:
import json
import pandas as pd
from tqdm import tqdm

In [ ]:
# Initialize lists
folder_names = []  # list of names of folders containing each dataset
dataset_names = []  # list of "title" fields from dataset-metadata.json
file_names = []  # list of all files in each folder except for dataset-metadata.json and zip files
licenses = []  # list of "licenses name" fields from dataset-metadata.json
descriptions = []  # list of "description" fields from dataset-metadata.json

In [ ]:
# Function to process each Kaggle dataset folder
def process_dataset_folder(folder_path):
    try:
        # Path to the dataset-metadata.json file
        metadata_path = os.path.join(folder_path, 'dataset-metadata.json')
        
        # Read the dataset-metadata.json file
        with open(metadata_path, 'r') as f:
            metadata = json.load(f)

        # Extract the required information
        dataset_name = metadata.get('title', '')
        license_name = metadata.get('licenses', [{}])[0].get('name', '')
        description = metadata.get('description', '')

        # List all files in the folder except for dataset-metadata.json and zip files
        files = [f for f in os.listdir(folder_path) if f != 'dataset-metadata.json' and not f.endswith('.zip')]

        # Store the information in the lists
        for file in files:
            folder_names.append(os.path.basename(folder_path))
            dataset_names.append(dataset_name)
            file_names.append(file)
            licenses.append(license_name)
            descriptions.append(description)
        
    except Exception as e:
        print(f"Error processing folder {folder_path}: {e}")

In [ ]:
# Iterate over each folder in the base directory
for folder_name in tqdm(os.listdir(base_directory), desc="Processing Kaggle Datasets"):
    folder_path = os.path.join(base_directory, folder_name)
    if os.path.isdir(folder_path):
        process_dataset_folder(folder_path)

In [ ]:
# Construct the DataFrame
data = {
    'Folder Name': folder_names,
    'Dataset Name': dataset_names,
    'File Name': file_names,
    'License': licenses,
    'Description': descriptions
}

df = pd.DataFrame(data)

# Adjust display options to show the complete DataFrame
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

In [ ]:
len(df)

#### 1.3 Datasets pre-processing

##### 1.3.1 Filter out csv files
- 12,533 -> 8,629

In [ ]:
# Extract file types (extensions) and make them lowercase
df['File Type'] = df['File Name'].apply(lambda x: os.path.splitext(x)[1].lower())

# Analyze the file types
file_type_counts = df['File Type'].value_counts().reset_index()
file_type_counts.columns = ['File Type', 'Count']

In [ ]:
file_type_counts

In [ ]:
# Filter the original df to include only CSV files
csv_df = df[df['File Type'] == '.csv']

# Reset the index of the filtered DataFrame
csv_df.reset_index(drop=True, inplace=True)

In [ ]:
len(csv_df)

##### 1.3.2 Filter out datasets w/ allowed licenses
- 8,629 -> 7,012

In [ ]:
# Analyze the license
license_counts = csv_df['License'].value_counts().reset_index()
license_counts.columns = ['License', 'Count']

In [ ]:
license_counts

In [ ]:
# Filter the csv_df for allowed licenses
not_allowed_licenses = [
    "unknown",
    "copyright-authors",
]

csv_df_licensed = csv_df[~csv_df['License'].isin(not_allowed_licenses)]

In [ ]:
len(csv_df_licensed)

##### 1.3.3 Filter out datasets w/ description
- 7,012 -> 6,520

In [ ]:
# Filter out datasets with descriptions
csv_df_desc = csv_df_licensed[csv_df_licensed['Description'].str.strip() != ""]

In [ ]:
len(csv_df_desc)

In [ ]:
display(csv_df_desc.head())

##### 1.3.4 Check dataset-table mapping
- 6,520 -> 2,357 (one-to-one mapping only: each dataset contains ONLY one table)

In [ ]:
# Group by Dataset Name and count the number of File Names for each Dataset
table_count_per_dataset = csv_df_desc.groupby('Dataset Name').size().reset_index(name='Table Count')

In [ ]:
table_count_per_dataset

In [ ]:
# Filter datasets that have only one table
one_table_per_dataset = table_count_per_dataset[table_count_per_dataset['Table Count'] == 1]

In [ ]:
len(one_table_per_dataset)

In [ ]:
# Join with the original dataframe `csv_df_desc` to get all other attributes
one_table_datasets = pd.merge(one_table_per_dataset, csv_df_desc, on='Dataset Name')

In [ ]:
display(one_table_datasets.head())

- 6,520 -> 4,163 (one-to-multiple mapping: each dataset contains MULTIPLE tables)

In [ ]:
# Filter datasets that have multiple tables
multi_table_per_dataset = table_count_per_dataset[table_count_per_dataset['Table Count'] > 1]

In [ ]:
multi_table_per_dataset['Table Count'].sum()

In [ ]:
# Join with the original dataframe `csv_df_desc` to get all other attributes
multi_table_datasets = pd.merge(multi_table_per_dataset, csv_df_desc, on='Dataset Name')

In [ ]:
multi_table_datasets

In [ ]:
len(multi_table_datasets)

In [ ]:
# !pip install chardet

import chardet

In [ ]:
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        raw_data = f.read(10000)  # Read only the first 10k bytes
    result = chardet.detect(raw_data)
    return result['encoding']

In [ ]:
def read_csv_with_multiple_encodings(file_path):
    encodings = ['utf-8', 'latin1', 'cp1252']
    detected_encoding = detect_encoding(file_path)
    encodings.insert(0, detected_encoding)  # Try detected encoding first
    for encoding in encodings:
        try:
            return pd.read_csv(file_path, nrows=0, encoding=encoding)  # Read only the header
        except Exception as e:
            continue
    return None  # If all attempts fail

In [ ]:
def check_schema_consistency(folder_name, base_directory):
    folder_path = os.path.join(base_directory, folder_name)
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv') and os.path.isfile(os.path.join(folder_path, f))]
    
    if not csv_files:
        return False

    schemas = []
    for file in csv_files:
        file_path = os.path.join(folder_path, file)
        try:
            df = read_csv_with_multiple_encodings(file_path)
            if df is not None:
                schemas.append(set(df.columns))
            else:
                print(f"Error reading {file_path}: Unable to decode with common encodings")
                return False
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
            return False

    # Check if all schemas are identical
    first_schema = schemas[0]
    for schema in schemas:
        if schema != first_schema:
            return False

    return True

In [ ]:
# Add a column "Schema Consistency" to `multi_table_datasets`
schema_consistency = []

for folder_name in tqdm(multi_table_datasets['Folder Name'].unique(), desc="Checking Schema Consistency"):
    consistency = check_schema_consistency(folder_name, base_directory)
    schema_consistency.append((folder_name, consistency))

In [ ]:
# Convert the schema_consistency list to a DataFrame
schema_consistency_df = pd.DataFrame(schema_consistency, columns=['Folder Name', 'Schema Consistency'])

# Join the schema consistency results with `multi_table_datasets`
multi_table_datasets = pd.merge(multi_table_datasets, schema_consistency_df, on='Folder Name')

In [ ]:
multi_table_datasets['Schema Consistency'].value_counts()

In [ ]:
multi_table_same_schema = multi_table_datasets[multi_table_datasets['Schema Consistency'] == True]
multi_table_diff_schema = multi_table_datasets[multi_table_datasets['Schema Consistency'] == False]

In [ ]:
# Save the datasets to CSV files
one_table_datasets.to_csv('one_table_datasets.csv', index=False)
multi_table_same_schema.to_csv('multi_table_same_schema.csv', index=False)
multi_table_diff_schema.to_csv('multi_table_diff_schema.csv', index=False)

#### 1.4 Visualize # of datasets after different stages of filtering

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Data for the stacked bar chart
categories = ['One-to-One Mapping', 'One-to-Multiple Mapping']
values_one_to_one = [2357, 0]
values_multiple_consistent = [0, 1461]
values_multiple_inconsistent = [0, 2702]

# Data for the funnel chart
stages = [
    "Total Datasets from Kaggle",  # df
    "Filter by CSV",  # csv_df
    "Filter by Licensing",  # csv_df_licensed
    "Filter by Descriptions",  # csv_df_desc
]
counts = [12533, 8629, 7012, 6520]

# Create subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plotting the funnel chart on the first subplot
ax1.barh(stages, counts)

# Adding labels on the bars for the funnel chart
for index, value in enumerate(counts):
    ax1.text(value, index, str(value), va='center')

# Setting title and labels for the first plot
ax1.set_title('Funnel Chart of Dataset Filtering Stages')
ax1.set_xlabel('Number of Files')
ax1.set_ylabel('Filtering Stages')
ax1.invert_yaxis()  # Reverse the order of stages for a funnel effect

# Plotting the stacked bar chart on the second subplot
p1 = ax2.bar(categories, values_one_to_one, label='One-to-One Mapping')
p2 = ax2.bar(categories, values_multiple_consistent, bottom=values_one_to_one, label='One-to-Multiple (Consistent Schema)')
p3 = ax2.bar(categories, values_multiple_inconsistent, bottom=[i+j for i,j in zip(values_one_to_one, values_multiple_consistent)], label='One-to-Multiple (Inconsistent Schema)')

# Add text labels on the bars for the stacked bar chart
def add_labels(bars, ax):
    for bar in bars:
        yval = bar.get_height()
        if yval > 0:
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_y() + yval/2, int(yval), ha='center', va='center', color='white')

add_labels(p1, ax2)
add_labels(p2, ax2)
add_labels(p3, ax2)

# Setting title and labels for the second plot
ax2.set_title('Dataset-Table Mapping')
ax2.set_xlabel('Mapping Type')
ax2.set_ylabel('Number of Tables')
ax2.legend(loc='upper left')

plt.tight_layout()
plt.show()

### 2. Datasets Metadata Enhancement

#### 2.1 Generate keywords & tasks for all datasets

In [ ]:
import pandas as pd
import os
import json
from tqdm import tqdm
import chardet

In [ ]:
# Load CSV files into dataframe
one_table_datasets = pd.read_csv('one_table_datasets.csv')
multi_table_same_schema = pd.read_csv('multi_table_same_schema.csv')
multi_table_diff_schema = pd.read_csv('multi_table_diff_schema.csv')

In [ ]:
from dotenv import load_dotenv
from openai import OpenAI 

In [ ]:
# Set the API key and model name
load_dotenv()

MODEL="gpt-4o"
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
def generate_keywords_and_queries(description, dataset_name, filenames=None):
    if filenames:
        filenames_str = "\n- ".join(filenames)
        
        prompt = f"""
            Given a dataset that contains CSV files with the below file names:
                - {filenames_str}
            And the description of the dataset provided below:
                {description}

            Generate a dictionary in JSON format with the CSV file names as keys and as values a list of 4 semantically distinct data content keywords that describe the expected content of each CSV file but do not describe specific analytic tasks possible with the file.
            For each CSV file, also add a list of 3 semantically distinct analytics task sentences that can be performed with the described CSV file, e.g. develop ML model to predict XYZ.

            Example response format:
            {{
                "csv file 1": {{
                    "keywords": ["keyword1", "keyword2", "keyword3", "keyword4"],
                    "task_queries": ["Task query 1", "Task query 2", "Task query 3"]
                }},
                "csv file 2": {{
                    "keywords": ["keyword1", "keyword2", "keyword3", "keyword4"],
                    "task_queries": ["Task query 1", "Task query 2", "Task query 3"]
                }}
            }}
        """
    else:
        prompt = f"""
            Based on the dataset description provided below, generate a dictionary in JSON format with the dataset name as key and as values a list of 4 semantically distinct data content keywords that describe the expected content of the dataset but do not describe specific analytic tasks possible with the data. 
            Also, add a list of 3 semantically distinct analytics task sentences that can be performed with the described dataset, e.g. develop ML model to predict XYZ.
            If it is hard to complete the task, return an empty dictionary instead.

            Dataset Description:
            "{description}"

            Example response format:
            {{
                "{dataset_name}": {{
                    "keywords": ["keyword1", "keyword2", "keyword3", "keyword4"],
                    "task_queries": ["Task query 1", "Task query 2", "Task query 3"]
                }}
            }}
        """

    response = client.chat.completions.create(
        model=MODEL,
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": "You are a helpful assistant designed generate keywords and task-based queries for tables."},
            {"role": "user", "content": prompt}
        ]
        
    )

    keywords_and_queries = response.choices[0].message.content
    
    try:
        return json.loads(keywords_and_queries)
    except json.JSONDecodeError:
        return {"error": "Failed to decode JSON response"}

In [ ]:
def process_datasets(one_table_datasets, multi_table_same_schema, multi_table_diff_schema):
    results = []

    # Process one_table_datasets and multi_table_same_schema
    for df, table_type in zip([one_table_datasets, multi_table_same_schema], ['one_table', 'multi_table_same_schema']):
        for dataset_name, group in tqdm(df.groupby('Dataset Name'), desc=f"Processing {table_type} datasets"):
            description = group.iloc[0]['Description']
            keywords_and_queries = generate_keywords_and_queries(description, dataset_name)
            for _, row in group.iterrows():
                file_name = row['File Name']
                keywords = keywords_and_queries.get(dataset_name, {}).get('keywords', [])
                task_queries = keywords_and_queries.get(dataset_name, {}).get('task_queries', [])
                results.append((row['Dataset Name'], file_name, keywords, task_queries, table_type))
    
    # Process multi_table_diff_schema
    for index, row in tqdm(multi_table_diff_schema.iterrows(), desc="Processing multi_table_diff_schema datasets", total=multi_table_diff_schema.shape[0]):
        dataset_name = row['Dataset Name']
        description = row['Description']
        csv_file = row['File Name']
        
        keywords_and_queries = generate_keywords_and_queries(description, dataset_name, [csv_file])
        keywords = keywords_and_queries.get(csv_file, {}).get('keywords', [])
        task_queries = keywords_and_queries.get(csv_file, {}).get('task_queries', [])
        results.append((dataset_name, csv_file, keywords, task_queries, 'multi_table_diff_schema'))

    return results

In [ ]:
def create_test_samples(one_table_datasets, multi_table_same_schema, multi_table_diff_schema, n=5):
    one_table_sample = one_table_datasets.sample(n=min(n, len(one_table_datasets)))
    multi_table_same_schema_sample = multi_table_same_schema.sample(n=min(n, len(multi_table_same_schema)))
    multi_table_diff_schema_sample = multi_table_diff_schema.sample(n=min(n, len(multi_table_diff_schema)))
    
    return one_table_sample, multi_table_same_schema_sample, multi_table_diff_schema_sample

In [ ]:
# Create 20 test samples
one_table_sample, multi_table_same_schema_sample, multi_table_diff_schema_sample = create_test_samples(one_table_datasets, multi_table_same_schema, multi_table_diff_schema, n=20)

In [ ]:
# Process the test samples
test_results = process_datasets(one_table_sample, multi_table_same_schema_sample, multi_table_diff_schema_sample)

In [ ]:
# Convert the results to a DataFrame
test_results_df = pd.DataFrame(test_results, columns=['Dataset Name', 'CSV File', 'Keywords', 'Task Queries', 'Table Type'])

In [ ]:
# Join the `test_results_df` back to the corresponding original DataFrames
one_table_sample = pd.merge(one_table_sample, test_results_df[test_results_df['Table Type'] == 'one_table'], 
                            left_on=['Dataset Name', 'File Name'], right_on=['Dataset Name', 'CSV File'], how='left')

multi_table_same_schema_sample = pd.merge(multi_table_same_schema_sample, test_results_df[test_results_df['Table Type'] == 'multi_table_same_schema'], 
                                          left_on=['Dataset Name', 'File Name'], right_on=['Dataset Name', 'CSV File'], how='left')

multi_table_diff_schema_sample = pd.merge(multi_table_diff_schema_sample, test_results_df[test_results_df['Table Type'] == 'multi_table_diff_schema'], 
                                          left_on=['Dataset Name', 'File Name'], right_on=['Dataset Name', 'CSV File'], how='left')

In [ ]:
# Union the DataFrames
union_df_sample = pd.concat([one_table_sample, multi_table_same_schema_sample, multi_table_diff_schema_sample], ignore_index=True)

In [ ]:
# Process all the datasets
results = process_datasets(one_table_datasets, multi_table_same_schema, multi_table_diff_schema)

#### 2.2 Add example rows (in markdown format) to each dataset

In [ ]:
import pandas as pd
import os
import json
from tqdm import tqdm
import chardet
import csv
import signal
import time

In [ ]:
# Load CSV files into dataframe
one_table_datasets = pd.read_csv('one_table_datasets.csv')
multi_table_same_schema = pd.read_csv('multi_table_same_schema.csv')
multi_table_diff_schema = pd.read_csv('multi_table_diff_schema.csv')

In [ ]:
# Helper functions

class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException

# Set the signal handler for the timeout
signal.signal(signal.SIGALRM, timeout_handler)

# Function to detect file encoding
def detect_encoding(file_path):
    if not os.path.isfile(file_path):
        print(f"Path is not a file: {file_path}")
        return None
    
    with open(file_path, 'rb') as f:
        raw_data = f.read(10000)  # Read only the first 10k bytes
    result = chardet.detect(raw_data)
    return result['encoding']

# Function to read CSV with multiple encodings
def read_csv_with_multiple_encodings(file_path):
    encodings = ['utf-8', 'latin1', 'cp1252', 'utf-16']
    detected_encoding = detect_encoding(file_path)
    if detected_encoding and detected_encoding.lower() not in encodings and detected_encoding.lower() != 'ascii':
        encodings.insert(0, detected_encoding)  # Try detected encoding first
    
    for encoding in encodings:
        try:
            # Set the timeout for reading a file
            signal.alarm(5)  # Timeout in seconds
            df = pd.read_csv(file_path, encoding=encoding, sep=None, engine='python', on_bad_lines='skip')  # Skip bad lines
            signal.alarm(0)  # Reset the alarm
            return df
        except TimeoutException:
            print(f"Timeout while reading file '{file_path}' with encoding '{encoding}'")
        except Exception as e:
            continue
    return None  # If all attempts fail

# Function to format example rows into markdown
def format_table_markdown(nested_array, n_rows=10):
    # The first row of the array is the header
    headers = nested_array[0]
    # The rest of the array are the data rows
    data_rows = nested_array[1:n_rows]

    # Start building the Markdown table
    markdown = "| " + " | ".join(str(header) for header in headers) + " |\n"
    markdown += "| " + " | ".join(["---"] * len(headers)) + " |\n"

    # Add data rows
    for row in data_rows:
        markdown += "| " + " | ".join(str(item) for item in row) + " |\n"
    return markdown

In [ ]:
# Function to add example rows to the DataFrames
def add_example_rows_to_dataframe(df):
    example_rows = []
    error_count = 0
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Adding example rows"):
        file_path = os.path.join('kaggle', row['Folder Name'], row['File Name'])
        start_time = time.time()
        # print(f"Processing file {idx + 1}/{len(df)}: {file_path}")
        
        try:
            signal.alarm(10)  # Timeout for the entire file processing
            df_sample = read_csv_with_multiple_encodings(file_path)
            signal.alarm(0)  # Reset the alarm

            if df_sample is None:
                error_count += 1
                print(f"Failed to read file: {file_path}")
                example_rows.append([])
            elif len(df_sample) == 0:
                error_count += 1
                print(f"Empty DataFrame for file: {file_path}")
                example_rows.append([])
            else:
                example = format_table_markdown([df_sample.columns] + df_sample.head(10).values.tolist())
                example_rows.append(example)
        
        except TimeoutException:
            error_count += 1
            print(f"Timeout while processing file '{file_path}'")
            example_rows.append([])  # Append an empty list if processing times out
        except Exception as e:
            error_count += 1
            print(f"Unexpected error processing file '{file_path}': {e}")
            example_rows.append([])  # Append an empty list if there's any other error

        end_time = time.time()
        # print(f"Processed file {idx + 1}/{len(df)} in {end_time - start_time:.2f} seconds")
        
        # Explicitly flush the output to ensure logs are written in real-time
        import sys
        sys.stdout.flush()

    df['Example Rows'] = example_rows

    print(f"Total number of tables that encountered errors: {error_count}")
    return df

In [ ]:
# Add example rows in markdown format to the three DataFrames
one_table_datasets = add_example_rows_to_dataframe(one_table_datasets)
multi_table_same_schema = add_example_rows_to_dataframe(multi_table_same_schema)
multi_table_diff_schema = add_example_rows_to_dataframe(multi_table_diff_schema)

#### 2.3 Generate time & geographic granularity for all datasets

In [ ]:
def generate_granularities(filename, table_markdown):
    prompt = f"""
        Given a table with file name '{filename}' with its header and first few example records: {table_markdown}, determine the most likely geographic or temporal granularity reflected in the dataset.
        Select the temporal granularity from the following options: Year, Quarter, Month, Week, Day, Hour, Minute, or Second. For the geographic granularity, choose from: Continent, Country, State/Province, County/District, City, or Zip Code/Postal Code. 
        Identify the granularities if reflected based on the table provided, or leave empty if it cannot be inferred from the table.

        Generate a dictionary in JSON format with two keys: 'time_granularity' and 'geo_granularity'.
        Example response format:
        {{
            "time_granularity": time_granularity,
            "geo_granularity": geo_granularity
        }}
    """
    
    try:
        response = client.chat.completions.create(
            model=MODEL,
            response_format={ "type": "json_object" },
            messages=[
                {"role": "system", "content": "You are a helpful assistant designed generate granularities for tables."},
                {"role": "user", "content": prompt}
            ]

        )

        granularities = response.choices[0].message.content 
        
        return json.loads(granularities)
    except json.JSONDecodeError:
        return {"error": "Failed to decode JSON response"}

In [ ]:
# Load CSV files into dataframe
one_table_datasets = pd.read_csv('one_table_datasets_with_rows.csv')
multi_table_same_schema = pd.read_csv('multi_table_same_schema_with_rows.csv')
multi_table_diff_schema = pd.read_csv('multi_table_diff_schema_with_rows.csv')

In [ ]:
# Function to process the DataFrame and generate granularities
def process_and_generate_granularities(df, df_name):
    results = []

    for index, row in tqdm(df.iterrows(), desc=f"Processing {df_name}", total=df.shape[0]):
        dataset_name = row['Dataset Name']
        table_num = row['Table Count']
        folder_name = row['Folder Name']
        csv_file = row['File Name']
        example_rows = row['Example Rows']
        description = row['Description']

        try:
            granularities = generate_granularities(csv_file, example_rows)
            time_granularity = granularities.get('time_granularity', '')
            geo_granularity = granularities.get('geo_granularity', '')
            results.append((dataset_name, table_num, folder_name, csv_file, example_rows, time_granularity, geo_granularity, description, df_name))
        except Exception as e:
            print(f"Error generating granularities from file '{csv_file}': {e}")
            continue
    
    # Create a DataFrame from the results and save it to a CSV file
    results_df = pd.DataFrame(results, columns=['Dataset Name', 'Table Count', 'Folder Name', 'File Name', 'Example Rows', 'Time Granularity', 'Geographic Granularity', 'Description', 'Schema Type'])
    results_df.to_csv(f'{df_name}_with_granu.csv', index=False)
    print(f"Saved results to {df_name}_with_granu.csv") 

In [ ]:
process_and_generate_granularities(one_table_datasets, 'one_table_datasets')

In [ ]:
process_and_generate_granularities(multi_table_same_schema, 'multi_table_same_schema')

In [ ]:
process_and_generate_granularities(multi_table_diff_schema, 'multi_table_diff_schema')

In [ ]:
# Load CSV files into dataframe
one_table_datasets_with_granu = pd.read_csv('one_table_datasets_with_granu.csv')
multi_table_same_schema_with_granu = pd.read_csv('multi_table_same_schema_with_granu.csv')
multi_table_diff_schema_with_granu = pd.read_csv('multi_table_diff_schema_with_granu.csv')

In [ ]:
# Concatenate the 3 dataframes
granu_df = pd.concat([one_table_datasets_with_granu, multi_table_same_schema_with_granu, multi_table_diff_schema_with_granu], ignore_index=True)

In [ ]:
# one_table_datasets: 2,357 -> 2,355
# multi_table_same_schema: 1,461 -> 1,461
# multi_table_diff_schema: 2,702 -> 2,701
table_counts = filtered_granu_df.groupby('Schema Type').size().reset_index(name='Count')
table_counts

In [ ]:
filtered_granu_df.head()

#### 2.4 Add # of columns & # of rows to each dataset

In [ ]:
# Function to count the number of columns in the Example Rows column
def count_columns(example_rows):
    if pd.isna(example_rows) or example_rows == '':
        return 0
    headers = example_rows.split('\n')[0].strip('|').split('|')
    return len(headers)

In [ ]:
filtered_granu_df['Number of Columns'] = filtered_granu_df['Example Rows'].apply(count_columns)

In [ ]:
# Function to count the number of rows in the original CSV file
def count_rows(folder_name, file_name):
    file_path = os.path.join('kaggle', folder_name, file_name)
    df = read_csv_with_multiple_encodings(file_path)
    if df is not None:
        return len(df)

In [ ]:
tqdm.pandas(desc="Counting rows")

filtered_granu_df['Number of Rows'] = filtered_granu_df.progress_apply(
    lambda row: count_rows(row['Folder Name'], row['File Name']), axis=1
)

In [ ]:
# Filter out rows where 'Example Rows' is empty list or 'Number of Rows' is NA
filtered_granu_df = filtered_granu_df[(filtered_granu_df['Example Rows'] != '[]')]
filtered_granu_df = filtered_granu_df.dropna(subset=['Number of Rows'])

In [ ]:
# Convert 'Number of Rows' to int
filtered_granu_df['Number of Rows'] = filtered_granu_df['Number of Rows'].astype(int)

In [ ]:
filtered_granu_df.to_csv('granu_with_column_row_counts_filtered.csv', index=False)

In [ ]:
len(filtered_granu_df)

In [ ]:
filtered_granu_df.head()

#### 2.5 Add popularity (`totalDownloads`), tags (`keywords`) & usability rating (`usabilityRating`) to each dataset

In [ ]:
def extract_metadata(folder_name):
    file_path = os.path.join('kaggle', folder_name, 'dataset-metadata.json')
    if os.path.exists(file_path):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                metadata = json.load(f)
                total_downloads = metadata.get('totalDownloads', None)
                keywords = metadata.get('keywords', [])
                formatted_keywords = str(keywords) if keywords else "[]"
                usability_rating = metadata.get('usabilityRating', None)
                return total_downloads, formatted_keywords, usability_rating
        except Exception as e:
            print(f"Error reading metadata for folder '{folder_name}': {e}")
            return None, "[]", None
    else:
        print(f"Metadata file not found for folder '{folder_name}'")
        return None, "[]", None

In [ ]:
filtered_granu_df[['Popularity', 'Tags', 'Usability Rating']] = filtered_granu_df.apply(
    lambda row: pd.Series(extract_metadata(row['Folder Name'])),
    axis=1
)

#### 2.6 Add file size to each dataset

In [ ]:
def get_file_size(folder_name, file_name):
    file_path = os.path.join('kaggle', folder_name, file_name)
    if os.path.exists(file_path):
        try:
            size = os.path.getsize(file_path)
            return size
        except Exception as e:
            print(f"Error getting size for file '{file_name}' in folder '{folder_name}': {e}")
            return None
    else:
        print(f"File not found: {file_path}")
        return None

In [ ]:
filtered_granu_df['File Size (bytes)'] = filtered_granu_df.apply(
    lambda row: get_file_size(row['Folder Name'], row['File Name']),
    axis=1
)

In [ ]:
filtered_granu_df.to_csv('eval_dataset_with_metadata.csv', index=False)

In [ ]:
len(filtered_granu_df)

#### 2.7 Add keywords & task queries to each dataset

In [ ]:
task_keyword_queries = pd.read_csv('task_keyword_queries_kaggle.csv')

In [ ]:
# Change the value in "Table Type" column from "one_table" to "one_table_datasets"
task_keyword_queries['Table Type'] = task_keyword_queries['Table Type'].replace('one_table', 'one_table_datasets')

In [ ]:
task_keyword_df = pd.merge(
    filtered_granu_df,
    task_keyword_queries[['Dataset Name', 'File Name', 'Keywords', 'Task Queries', 'Table Type']],
    left_on=['Dataset Name', 'File Name', 'Schema Type'],
    right_on=['Dataset Name', 'File Name', 'Table Type'],
    how='left'
)

In [ ]:
# Drop the Table Type column from task_keyword_queries
task_keyword_df = task_keyword_df.drop(columns=['Table Type'])

In [ ]:
task_keyword_df.to_csv('eval_dataset_with_task_keyword.csv', index=False)

In [ ]:
task_keyword_df.head()

#### 2.8 Generate metadata queries for ALL metadata available
- Availble metadata fields:
    - Time Granularity
    - Geographic Granularity
    - Number of Columns
    - Number of Rows
    - Popularity
    - Usability Rating
    - Tags
    - File Size (bytes)

In [ ]:
import pandas as pd
import os
import ast
from tqdm import tqdm
from dotenv import load_dotenv
from openai import OpenAI 
import instructor
from pydantic import BaseModel, ValidationError, conlist

In [ ]:
task_keyword_df = pd.read_csv('eval_dataset_with_task_keyword.csv')

In [ ]:
# Set the API key, model name, and clients
load_dotenv()

MODEL="gpt-4o"
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
instructor_client = instructor.from_openai(client)

In [ ]:
class MetadataQuery(BaseModel):
    queries: conlist(str, min_length=1)

In [ ]:
# Initialize token usage counters
total_tokens_sum = 0
prompt_tokens_sum = 0
completion_tokens_sum = 0

In [ ]:
def generate_metadata_queries(task_queries, metadata):
    global total_tokens_sum, prompt_tokens_sum, completion_tokens_sum
    
    prompt_template = """
        Given the analytical task {task_query} and the available dataset metadata: {metadata}, 
        generate a list with one natural language metadata query for each metadata attribute that expresses the dataset requirement for that attribute needed to implement the given task.

        Output a list of metadata queries that are as specific and concise as possible in the specification. Queries should be within but far from the bound of the given metadata attribute value.
        So, with the metadata 'Number of Rows' equals to 4836, the corresponding query could be 'The dataset should contain at least 2000 rows', but could not be 'The dataset should contain at least 10000 rows'.

        For example, for the task 'Identify patterns in the trading volume of Adobe's stock' and metadata {{"Tags": ["Trading", "Forecasting"], "Time Granularity": "Day", "Number of Columns": 7, "Number of Rows": 3563}}, 
        this could yield: ['The dataset should have the trading tag', 'I need a dataset with data on minimum daily level', 'the dataset should contain roughly 5 columns', 'the dataset should contain at least 2000 rows']
        
        Only generate a list of metadata queries, excluding any introductory phrases and focusing exclusively on the tasks themselves.
    """
    
    metadata_queries = []
    
    # Remove metadata fields with NA values
    metadata = {k: v for k, v in metadata.items() if pd.notna(v)}
    
    for task_query in task_queries:
        prompt = prompt_template.format(task_query=task_query, metadata=metadata)
        
        try:
            response, completion = instructor_client.chat.completions.create_with_completion(
                model=MODEL,
                response_model=MetadataQuery,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant designed generate metadata queries given an analytical task and the available dataset metadata."},
                    {"role": "user", "content": prompt}
                ]
            )
            
            # Record token usage details
            token_usage = completion.usage
            total_tokens_sum += token_usage.total_tokens
            prompt_tokens_sum += token_usage.prompt_tokens
            completion_tokens_sum += token_usage.completion_tokens
            
            # Extract and validate the response
            extracted_queries = MetadataQuery(**response.__dict__)
            metadata_queries.append(extracted_queries.queries)
        except ValidationError as ve:
            print(f"ValidationError parsing response: {ve}")
            metadata_queries.append([])
        except Exception as e:
            print(f"Error generating metadata queries: {e}")
            metadata_queries.append([])
    
    return metadata_queries

In [ ]:
def process_metadata_queries(row):
    task_queries = ast.literal_eval(row['Task Queries'])
    metadata = {
        "Tags": row['Tags'],
        "Time Granularity": row['Time Granularity'],
        "Geographic Granularity": row['Geographic Granularity'],
        "Number of Columns": row['Number of Columns'],
        "Number of Rows": row['Number of Rows']
    }

    return generate_metadata_queries(task_queries, metadata)

In [ ]:
# Apply the function to the DataFrame and create the "Metadata Queries" column
tqdm.pandas(desc="Generating metadata queries")
task_keyword_df['Metadata Queries'] = task_keyword_df.progress_apply(process_metadata_queries, axis=1)

In [ ]:
print(f"Total Tokens Used: {total_tokens_sum}")
print(f"Prompt Tokens Used: {prompt_tokens_sum}")
print(f"Completion Tokens Used: {completion_tokens_sum}")

In [ ]:
task_keyword_df.to_csv('eval_dataset_final.csv', index=False)

### 3. Evaluation Dataset Split (Database Level): Stratified Sampling by Database Size

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
df = pd.read_csv('eval_data_all.csv')

In [ ]:
# Step 1: Calculate table count per database
table_counts = df['database_name'].value_counts().to_dict()
df['table_count'] = df['database_name'].map(table_counts)

In [ ]:
# Step 2: Categorize databases by table count
def categorize_database(size):
    if size == 1:
        return 'small'
    elif 2 <= size <= 10:
        return 'medium'
    else:
        return 'large'

df['db_size_category'] = df['table_count'].apply(categorize_database)

In [ ]:
# Step 3: Stratified split into train+val (60%) and test (40%) on database level
# Get unique databases and their categories
unique_databases = df[['database_name', 'db_size_category']].drop_duplicates()

train_val_dbs, test_dbs = train_test_split(
    unique_databases, test_size=0.4, stratify=unique_databases['db_size_category'], random_state=42
)

In [ ]:
# Step 4: Create the train+validation (60%) and test (40%) datasets
train_val_df = df[df['database_name'].isin(train_val_dbs['database_name'])]
test_df = df[df['database_name'].isin(test_dbs['database_name'])]

In [ ]:
# Step 5: Further split train+validation into training (40%) and validation (20%)
train_dbs, val_dbs = train_test_split(
    train_val_dbs, test_size=0.333, stratify=train_val_dbs['db_size_category'], random_state=42
)

train_df = train_val_df[train_val_df['database_name'].isin(train_dbs['database_name'])]
val_df = train_val_df[train_val_df['database_name'].isin(val_dbs['database_name'])]

In [ ]:
# Step 6: Save the split data into CSV files
train_df.to_csv('eval_data_train.csv', index=False)
val_df.to_csv('eval_data_validation.csv', index=False)
test_df.to_csv('eval_data_test.csv', index=False)

In [ ]:
# Summarize the final splits: # of databases & tables in each split
split_summary = {
    'Split': ['Training', 'Validation', 'Test'],
    'Databases': [
        train_df['database_name'].nunique(),
        val_df['database_name'].nunique(),
        test_df['database_name'].nunique()
    ],
    'Tables': [
        train_df.shape[0],
        val_df.shape[0],
        test_df.shape[0]
    ]
}

split_summary_df = pd.DataFrame(split_summary)
split_summary_df